# breast_cancer resuelto SIN la librería sklearn

## Normalizando los datos

In [5]:
import numpy as np

class StandardScalerManual:
    def __init__(self):
        self.mean_ = None
        self.scale_ = None
    
    def fit(self, X):
        """Calcula la media y desviación estándar de X."""
        self.mean_ = np.mean(X, axis=0)
        self.scale_ = np.std(X, axis=0)
        return self
    
    def transform(self, X):
        """Normaliza X usando la media y desviación estándar."""
        # Evitar división por cero
        scale = np.where(self.scale_ == 0, 1, self.scale_)
        return (X - self.mean_) / scale
    
    def fit_transform(self, X):
        """Ajusta el scaler a X y devuelve X normalizado."""
        return self.fit(X).transform(X)

class PerceptronManual:
    def __init__(self, learning_rate=0.01, n_iterations=1000, tol=1e-3, alpha=0.0001):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.tol = tol
        self.alpha = alpha  # Parámetro de regularización L2
        self.weights = None
        self.bias = None
        self.errors_ = []
        self.best_weights = None  # Guardar los mejores pesos
        self.best_bias = None     # Guardar el mejor bias
        self.best_error = float('inf')
    
    def fit(self, X, y):
        """Entrena el perceptrón usando el conjunto de datos X e y."""
        n_samples, n_features = X.shape
        
        # Inicialización He
        self.weights = np.random.randn(n_features) * np.sqrt(2. / n_features)
        self.bias = np.random.randn() * 0.1
        
        # Convertir a arrays de numpy si no lo son
        X = np.array(X)
        y = np.array(y)
        
        # Variables para early stopping
        patience = 5
        min_delta = 1e-4
        patience_counter = 0
        best_error = float('inf')
        
        # Entrenamiento
        for epoch in range(self.n_iterations):
            errors = 0
            
            # Learning rate adaptativo con decaimiento más suave
            current_lr = self.learning_rate / (1 + epoch * 0.005)
            
            # Crear índices aleatorios para shuffle
            indices = np.random.permutation(n_samples)
            X_shuffled = X[indices]
            y_shuffled = y[indices]
            
            for idx, x_i in enumerate(X_shuffled):
                # Predicción
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = 1 if linear_output > 0 else 0
                
                # Actualización de pesos si hay error
                if y_predicted != y_shuffled[idx]:
                    # Actualización con regularización L2
                    update = current_lr * (y_shuffled[idx] - y_predicted)
                    self.weights = self.weights * (1 - current_lr * self.alpha) + update * x_i
                    self.bias += update
                    errors += 1
            
            # Calcular error actual
            current_error = errors / n_samples
            self.errors_.append(current_error)
            
            # Guardar los mejores pesos si mejora el error
            if current_error < self.best_error:
                self.best_error = current_error
                self.best_weights = np.copy(self.weights)
                self.best_bias = self.bias
            
            # Early stopping con patience
            if current_error < best_error - min_delta:
                best_error = current_error
                patience_counter = 0
            else:
                patience_counter += 1
            
            if patience_counter >= patience:
                print(f"Early stopping en época {epoch+1}")
                break
            
            # Criterio de convergencia
            if current_error < self.tol:
                print(f"Convergencia alcanzada en época {epoch+1}")
                break
        
        # Usar los mejores pesos encontrados
        if self.best_weights is not None:
            self.weights = self.best_weights
            self.bias = self.best_bias
    
    def predict(self, X):
        """Realiza predicciones para el conjunto X."""
        linear_output = np.dot(X, self.weights) + self.bias
        return np.where(linear_output > 0, 1, 0)

def train_test_split_manual(X, y, test_size=0.2, random_state=None):
    """División manual de datos en conjuntos de entrenamiento y prueba."""
    if random_state is not None:
        np.random.seed(random_state)
    
    n_samples = len(X)
    n_test = int(n_samples * test_size)
    
    # Crear índices aleatorios
    indices = np.random.permutation(n_samples)
    test_indices = indices[:n_test]
    train_indices = indices[n_test:]
    
    # Dividir los datos
    X_train = X[train_indices]
    X_test = X[test_indices]
    y_train = y[train_indices]
    y_test = y[test_indices]
    
    return X_train, X_test, y_train, y_test

def accuracy_score_manual(y_true, y_pred):
    """Calcula la precisión del modelo."""
    return np.mean(y_true == y_pred)

def classification_report_manual(y_true, y_pred):
    """Genera un reporte de clasificación manual."""
    # Calcular verdaderos positivos, falsos positivos, etc.
    tp = np.sum((y_true == 1) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    
    # Calcular métricas
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    
    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'accuracy': accuracy
    }

In [6]:
import numpy as np
from sklearn.datasets import load_breast_cancer

# Cargar los datos
data = load_breast_cancer()
X, y = data.data, data.target

# Normalizar los datos
scaler = StandardScalerManual()
X_scaled = scaler.fit_transform(X)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split_manual(X_scaled, y, test_size=0.2, random_state=42)

# Probar diferentes valores de regularización
alphas = [0.0001, 0.001, 0.01]
best_accuracy = 0
best_alpha = None
best_model = None

for alpha in alphas:
    print(f"\nProbando alpha: {alpha}")
    perceptron = PerceptronManual(learning_rate=0.001, n_iterations=1000, tol=1e-3, alpha=alpha)
    perceptron.fit(X_train, y_train)
    
    y_pred = perceptron.predict(X_test)
    accuracy = accuracy_score_manual(y_test, y_pred)
    report = classification_report_manual(y_test, y_pred)
    
    print(f"Precisión: {accuracy:.4f}")
    print(f"Error final: {perceptron.errors_[-1]:.4f}")
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_alpha = alpha
        best_model = perceptron

print(f"\nMejor alpha: {best_alpha}")
print(f"Mejor precisión: {best_accuracy:.4f}")

# Imprimir reporte detallado del mejor modelo
y_pred = best_model.predict(X_test)
report = classification_report_manual(y_test, y_pred)
print("\nReporte de clasificación del mejor modelo:")
print(report)


Probando alpha: 0.0001
Early stopping en época 28
Precisión: 0.9646
Error final: 0.0329

Probando alpha: 0.001
Early stopping en época 20
Precisión: 0.9646
Error final: 0.0395

Probando alpha: 0.01
Early stopping en época 16
Precisión: 0.9558
Error final: 0.0285

Mejor alpha: 0.0001
Mejor precisión: 0.9646

Reporte de clasificación del mejor modelo:
{'precision': np.float64(0.971830985915493), 'recall': np.float64(0.971830985915493), 'f1_score': np.float64(0.971830985915493), 'accuracy': np.float64(0.9646017699115044)}
